In [ ]:
%load_ext autoreload
%autoreload 2

In [22]:
from bench_lib.evaluation import benchmark_results_folder

rfolder = benchmark_results_folder()
folders = ["gemma-3-4b-it", "gemma-3-12b-it", "gemma-3-27b-it_00", "qwen-2.5-vl", "gemini-2.5-pro-noschema"]
files = [str(rfolder / f / "toxicainment_videos_log.jsonl") for f in folders]

In [23]:
import duckdb

con = duckdb.connect()
query = f"SELECT * FROM read_json({files})"
df = con.sql(query).df()

In [ ]:
df.head(2)

In [ ]:
df.groupby("Model ID", as_index=False).agg(
    avg_total_runtime=("Total_Runtime", "mean"),
    avg_model_runtime=("Model_Runtime", "mean"),
    avg_tokens_generated=("Tokens_Generated", "mean"),
    avg_peak_memory_alloc=("Peak_Memory_Allocated", "mean"),
    avg_peak_memory_reserved=("Peak_Memory_Reserved", "mean"),
    avg_total_frames=("Total_Frames", "mean"),
).sort_values("avg_model_runtime", ascending=False).round(2)

In [ ]:
from bench_lib.evaluation import plot_runtime_side_by_side


fig = plot_runtime_side_by_side(
    df.query("`Model ID` == 'Qwen/Qwen2.5-VL-3B-Instruct'"),
    df.query("`Model ID` == 'Qwen/Qwen2.5-VL-72B-Instruct'"),
    title1="Qwen2.5-VL 3B",
    title2="Qwen2.5-VL 72B",
)
fig.savefig("imgs/runtime_scatter_qwenvl.png", dpi=300, bbox_inches="tight")

fig = plot_runtime_side_by_side(
    df.query("`Model ID` == 'google/gemma-3-4b-it'"),
    df.query("`Model ID` == 'google/gemma-3-27b-it'"),
    title1="Gemma-3 4B",
    title2="Gemma-3 27B",
)
fig.savefig("imgs/runtime_scatter_g3.png", dpi=300, bbox_inches="tight")


In [ ]:
df.query("`Model ID`.str.contains('gemini')")["Total_Runtime"].describe()